In [ ]:
import numpy as np

# Good intentions, wrong advice
## Don't follow these mantras, please - or at least, not in the way you think...

"Premature optimization is the root of all evil" (Knuth, Hoare - they blame each other)

"Make it work, Make it right, Make it fast" (Kent Beck, Agile Manifesto)


# These shouldn't be tradeoffs

Good design results in code that works right _and_ fast

It can be hard to undo bad design decisions (without rewriting from scratch)

Is this 'premature optimization'?

# Datatypes

Basic datatypes in Python (and numpy)

List vs np.ndarray

These have _very_ different performance profiles.  This is not a "Python thing" - these are fundamentally different data structures

https://en.wikipedia.org/wiki/Linked_list

https://en.wikipedia.org/wiki/Array_data_structure

https://en.wikipedia.org/wiki/Time_complexity

In [ ]:
# Let's write a few functions to compare performance

def reorder_list(x: list, idx: list):
    return [x[i] for i in idx]

In [ ]:
N = 100000

x_np = np.linspace(0.0,1.0,N)
x_list = list(x_np)

idx_r = range(N)

idx_np = np.array(idx_r)
idx_list = list(idx_r)

In [ ]:
# Use the %time and %timeit macros to explore execution times
# We'll cover more advanced profiling later

In [ ]:
%timeit _ = reorder_list(x_list, idx_list)

In [ ]:
%timeit _ = x_np[idx_np]

In [ ]:
# Just using numpy datatypes isn't enough - we need to use numpy idioms

def reorder_ndarray_naive(x: np.ndarray, idx: np.ndarray):
    out = np.empty(len(x))
    for i, i_src in enumerate(idx):
        out[i] = x[i_src]
    return out

In [ ]:
%timeit _ = reorder_ndarray_naive(x_np, idx_np)

### Time complexity - data access

In [ ]:
# Constant time O(1)

%timeit _ = x_np[100:800]
%timeit _ = x_np[100:80000]

In [ ]:
# Linear time O(n)

%timeit _ = x_list[100:800]
%timeit _ = x_list[100:80000]

In [ ]:
# Pathological numpy - use the idioms!
# Forces O(n)

%timeit _ = x_np[range(100,800)]
%timeit _ = x_np[range(100,80000)]

### Time complexity - append

Lists do much better here!

In [ ]:
%timeit _ = np.append(x_np,1.0)

In [ ]:
big_arr = np.zeros(N*10)

%timeit _ = np.append(big_arr,1.0)

In [ ]:
%timeit _ = [].append(1.0)

In [ ]:
real_list = [x for x in range(1000)]

%timeit _ = real_list.append(1.0)

In [ ]:
del real_list

# Python specifics

...or...

## Make Python Fast by Not Using Python

(Don't worry, you already do)

In [ ]:
import math

In [ ]:
%timeit math.exp(5.0)

In [ ]:
%timeit np.exp(5.0)

In [ ]:
%timeit for x in big_arr: math.exp(x)

In [ ]:
%timeit np.exp(big_arr)

## ufuncs

Functions in numpy (or derived from numpy) that operate over both scalars and arrays of arbitrary size

In [ ]:
def naive_function(x:float , y: float):
    return (x*y)/math.exp(x+y)

In [ ]:
naive_function(2.0,3.0)

In [ ]:
def naive_function_loop(x: list, y: list):
    out = []
    for xval,yval in zip(x,y):
        out.append(naive_function(xval, yval))
    return out

In [ ]:
x = np.random.normal(size=(1000,))
y = np.random.normal(size=(1000,))

In [ ]:
naive_function_loop(x,y)

In [ ]:
%timeit _ = naive_function_loop(x,y)

In [ ]:
def ufunc_function(x, y):
    return (x*y)/np.exp(x+y)

In [ ]:
ufunc_function(2.0,3.0)

In [ ]:
%timeit _ = ufunc_function(x,y)

In [ ]:
ufunc_function(x,y) == naive_function_loop(x,y)

In [ ]:
import pandas as pd

In [ ]:
x_pd = pd.Series(x)
y_pd = pd.Series(y)

ufunc_function(x_pd, y_pd)

In [ ]:
# This will work in a way that respects pandas indices

x_pd = pd.Series(x, index=range(500,500+len(x)))
y_pd = pd.Series(y)

ufunc_function(x_pd, y_pd).plot()

In [ ]:
# Our naive loop will throw this away - plenty of opportunities for bugs...

pd.Series(naive_function_loop(x_pd, y_pd)).plot()

In [ ]:
#  Thinking back to the (bad) mantras - how does this vectorized (ufunc) style address these concerns?

# Advanced Profiling (cProfile)

In [ ]:
import cProfile

In [ ]:
from autumn.tools.project import get_project

In [ ]:
p = get_project('covid_19', 'malaysia')

In [ ]:
m = p.build_model(p.param_set.baseline.to_dict())

In [ ]:
%time m.run()

In [ ]:
cProfile.run("m.run()", sort='cumtime')

In [ ]:
cProfile.run("m.run()", sort='tottime')

In [ ]:
build_options = {
    "enable_validation": False,
    "derived_outputs_idx_cache": m._derived_outputs_idx_cache
}

In [ ]:
%time m = p.build_model(p.param_set.baseline.to_dict(), build_options=build_options)

In [ ]:
%time m.run()

In [ ]:
p = get_project('sm_sir', 'malaysia')

In [ ]:
%time m = p.build_model(p.param_set.baseline.to_dict())

In [ ]:
%time m.run()

In [ ]:
cProfile.run("m.run()", sort='cumtime')

In [ ]:
cProfile.run("m.run()", sort='tottime')